In [ ]:
class Value:
    """ stores a single scalar value and its gradient """

    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        # internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op # the op that produced this node, for graphviz / debugging / etc

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += (other * self.data**(other-1)) * out.grad
        out._backward = _backward

        return out

    def relu(self):
        out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')

        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward

        return out

    def backward(self):

        # topological order all of the children in the graph
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = 1
        for v in reversed(topo):
            v._backward()

    def __neg__(self): # -self
        return self * -1

    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other): # self - other
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"
    


In [ ]:
w = Value(0.5)

In [ ]:
#  # Just an example value
for i in range(10):
    x = Value(4)
    y_pred = w * x
    y_true = Value(3)
    loss = (y_pred - y_true) ** 2
    loss.backward()  # This populates the gradients
    learning_rate = 0.0001
    w.data -= learning_rate * w.grad
    print(y_pred, loss, w)

In [ ]:
# a + b = 5

w1 = Value(.5)
w2 = Value(-.5)

encoded_a = Value(.1)
encoded_b = Value(-.1)

In [ ]:
for i in range(10):
    y_pred = w1*encoded_a + w2*encoded_b
    y_truth = Value(5)

    loss = (y_truth - y_pred) **2

    loss.backward()
    learning_rate = 0.1
    w1.data -= learning_rate * w1.grad
    w2.data -= learning_rate * w2.grad

    print(y_pred, loss, w1, w2)


# for emm I want to pridict a


In [ ]:
vocab = ".abcdefghijklmnopqrstuvwxyz"
stoi = { char:i for i,char in enumerate(vocab)}
print(stoi)

In [ ]:
e  = Value(stoi['e'])
m  = Value(stoi['m'])
a = Value(stoi['a'])

y = a

w_e = Value(.1)
w_m = Value(.2)

In [ ]:
#pass emm
for i in range(10):
    y_pred = e*w_e + m * w_m + m * w_m

    loss = (y - y_pred)**2
    loss.backward()

    learning_rate = .0001
    w_e -=  learning_rate * e.grad
    w_m  -= learning_rate * m.grad

    print(y_pred.data, loss.data, w_e.data, w_m.data)


In [ ]:
words = open("../names.txt", 'r').read().splitlines()

In [ ]:


X = []
Y = []
for w in words[:5]:
    context = [0,0,0]
    for char in w:
        X.append(context)
        Y.append(char)
        
        context = context[1:] + [stoi[char]]
         

In [ ]:
Xb =  [ list(map(Value, xrow)) for xrow in X] 
Xb


In [141]:
# Mean and standard deviation for the Gaussian distribution
mean = 0
std_dev = 1

# Initializing a 3x27 array with Gaussian distributed random numbers
w1 = np.random.normal(mean, std_dev, (3, 27))



In [148]:
loss = 0
for i, d in enumerate(Xb):
   weights =  list(map(Value,w1[:,i]))
   logits = weights[0]*d[0] + weights[1]*d[1] + weights[2]*d[2]
   loss += (logits.data - Value(stoi[Y[i]]))**2 
   
print(loss)




Value(data=21417.158512077247, grad=0)


In [146]:
Y

['e',
 'm',
 'm',
 'a',
 'o',
 'l',
 'i',
 'v',
 'i',
 'a',
 'a',
 'v',
 'a',
 'i',
 's',
 'a',
 'b',
 'e',
 'l',
 'l',
 'a',
 's',
 'o',
 'p',
 'h',
 'i',
 'a']